# Callback
- no Dash um callback é uma função que conecta componentes da interface do usuario (UI) com a logica da aplicação. Os callbacks são usados para criar interatividade, permitindo que a interface responda a eventos, como cliques de botões, seleções de menus suspensos, alterações de entrada de texto entre outros

__Logica:__
- Entrada(Input): Os componentes que desencadeiam o callback quando seu estado muda EX:(um botão é clicado)
- Saida(Output): Os componentes que são atualizados como resultado da função callback
- Função callback: A função Python que define a lógica para atualizar a saida com base na entrada.

In [1]:
#!pip install pandas_datareader

In [2]:
#Imports
import pandas as pd
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas_datareader.data as web
import datetime

C:\Users\Guilherme\AppData\Local\Temp\ipykernel_18800\715095563.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Guilherme\AppData\Local\Temp\ipykernel_18800\715095563.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
# https://stooq.com/
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2021, 2, 15)

df = web.DataReader(['SVAC','JNJ','PFE','MRNA','BNTX'],
                    'stooq', start=start, end=end)

df = df.stack().reset_index()

cores = ["black", "red", "blue", "orange"]


C:\Users\Guilherme\AppData\Local\Temp\ipykernel_18800\2023831232.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = web.DataReader(['SVAC','JNJ','PFE','MRNA','BNTX'],


In [4]:
df.head()

Attributes,level_0,Symbols,Close,High,Low,Open,Volume
0,2019-01-02,JNJ,114.9620,115.5310,113.6470,115.3150,8.480359e+06
1,2019-01-02,PFE,37.3441,37.5599,36.8123,37.2319,2.904233e+07
2,2019-01-02,MRNA,15.3300,15.4500,14.5100,14.9300,2.318094e+06
3,2019-01-03,JNJ,113.1380,115.4320,112.8140,115.3150,9.617137e+06
4,2019-01-03,PFE,36.2979,37.4203,36.2209,37.3343,3.132042e+07


In [5]:
app1 = dash.Dash(__name__)

app1.layout = html.Div(
    children=[
    #Dash component
    html.H1(id = 'output-titulo', children='Valor de Ação covid-19', style={}),

    dcc.Dropdown(id='meu-dropdown', multi=True,
                options=[{'label': x , 'value': x} for x in sorted(df.Symbols.unique())],
                value=['SVAC','JNJ','PFE','MRNA','BNTX'],
                style={'text-align':'center'}
                ),
                
                html.Br(),

                dcc.Graph(id='grafico-output', figure={}),

                dcc.RadioItems(id='cores-id',value='black',options=[{'label': c,'value':c} for c in cores]),
    ]
)






__Trabalhando o callback__

In [6]:
@app1.callback(
    [Output('grafico-output', 'figure'), Output('output-titulo', 'style')],
    [Input(component_id='cores-id', component_property='value'),
    Input(component_id='meu-dropdown', component_property='value')],
    prevent_initial_call=False # True : Não ativa o callback quando a pagina é atualizada
)

def muda_grafico(cor_escolhida, acao_escolhida):

    if len(acao_escolhida) == 0:
        return dash.no_update, {"color": cor_escolhida} # faz update em somente um componente quando a página é atualizada - nesse caso a cor e nao o grafico
    else:
        df_copy = df[df["Symbols"].isin(acao_escolhida)]
        fig = px.line(df_copy, x='level_0', y='Close', color = 'Symbols')


    return fig, {"color": cor_escolhida, 'text-align': 'center' }




In [7]:
if __name__ == "__main__":
    app1.run_server(debug=True, port=2, use_reloader=False)

In [8]:
df.head()

Attributes,level_0,Symbols,Close,High,Low,Open,Volume
0,2019-01-02,JNJ,114.9620,115.5310,113.6470,115.3150,8.480359e+06
1,2019-01-02,PFE,37.3441,37.5599,36.8123,37.2319,2.904233e+07
2,2019-01-02,MRNA,15.3300,15.4500,14.5100,14.9300,2.318094e+06
3,2019-01-03,JNJ,113.1380,115.4320,112.8140,115.3150,9.617137e+06
4,2019-01-03,PFE,36.2979,37.4203,36.2209,37.3343,3.132042e+07
